In [101]:
from SPM import subspace_power_method
import numpy as np
from PyMoments import kstat
from itertools import permutations
from scipy.optimize import minimize
from scipy.stats import uniform_direction
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
import math
import helper_functions
from itertools import product
%matplotlib inline

In [102]:
def cumulant_tensors(observeddata):
    I=observeddata.shape[1]
    sym_indices,b,c=helper_functions.symmetric_indices(I,4)
    fourth_cumulants=np.apply_along_axis(lambda x: kstat(observeddata,tuple(x)), 0, sym_indices)
    fourth_cumulant_dict={tuple(sym_indices[:,n]):fourth_cumulants[n] for n in range(len(fourth_cumulants))}
    all_indices=np.array([list(i) for i in product(range(I), range(I),range(I),range(I))])
    values=np.apply_along_axis(lambda x:fourth_cumulant_dict[tuple(np.sort(x))],1,all_indices)
    fourth_order_kstats=values.reshape(I,I,I,I)
    
    sym_indices_2,b,c=helper_functions.symmetric_indices(I,2)
    second_cumulants=np.apply_along_axis(lambda x: kstat(observeddata,tuple(x)), 0, sym_indices_2)
    second_cumulant_dict={tuple(sym_indices_2[:,n]):second_cumulants[n] for n in range(len(second_cumulants))}
    all_indices_2=np.array([list(i) for i in product(range(I), range(I))])
    values=np.apply_along_axis(lambda x:second_cumulant_dict[tuple(np.sort(x))],1,all_indices_2)
    second_order_kstats=values.reshape(I,I)
    return second_order_kstats,fourth_order_kstats

In [103]:
# generate simulated data
# generate the sources, all but the last one are exponential distribution with parameter one and the last one is standard Gaussian
def observeddata(samplesize,I,J):
    sourcelist=[]
    for i in range(J-1):
        sourcelist.append(np.random.exponential(1,samplesize))
    sourcelist.append(np.random.normal(0,1,samplesize))
    sourcedata=np.transpose(np.vstack(tuple(sourcelist)))

    # generate a random mixing matrix with columns having norm 1
    uniformsphere=uniform_direction(I)
    A=np.transpose(uniformsphere.rvs(J))

    # mix the sources
    observeddata=np.transpose(A @ np.transpose(sourcedata))
    return A,observeddata


def recovermixingmatrix(I,J,observeddata):
    # generate the second and fourth cumulant tensors
    second_order_kstats,fourth_order_kstats=cumulant_tensors(observeddata)
    # recover first J-1 columns via tensor decomposition
    cols,lambdas=subspace_power_method(fourth_order_kstats,n=4,d=I,r=J-1)
    def returnmindistancebewteenvectors(cols):
        def distancebewteenvectors(v1,v2):
            v1=v1.reshape(-1,1)
            v2=v2.reshape(-1,1)
            M=v1@np.transpose(v1)-v2@np.transpose(v2)
            return np.sum(M*M)
        lens=cols.shape[1]
        error=1
        for i in range(lens):
            for j in range(i+1,lens):
                error=min(error,distancebewteenvectors(cols[:,i],cols[:,j]))
        return error
    step=0
    while returnmindistancebewteenvectors(cols)<0.1 and step<100:
        cols,lambdas=subspace_power_method(fourth_order_kstats,n=4,d=I,r=J-1)
        step+=1
    
    # now we use second cumulant to find the last column of the mixing matrix 
    # by minimizing the distance bewteen a matrix in the linear span of the second cumulant matrix 
    # and all the rank 1 matrices M_1,...M_{J-1} obtained from the first J-1 columns of A
    rank1_matrixlist=[]
    for i in range(J-1):
        vector=cols[:,i].reshape(-1,1)
        matrix=vector@ np.transpose(vector)
        rank1_matrixlist.append(matrix)
    
    #function returning the distance between second_order_kstats-l_1*M_1-...-l_{J-1}*M_{J-1} and v\otimes v
    # input is the vector (l|v)
    def sumofsquare(x):
        l=x[:J]
        v=x[J:]
        v=v.reshape(-1,1)
        n=len(l)
        M=second_order_kstats
        for i in range(n-1):
            M=M-l[i]*rank1_matrixlist[i]
        M=M-l[-1]*v@np.transpose(v)
        return np.sum(M*M)
    #initialize a random start point
    l0= np.random.randn(J).reshape(-1,1)
    v0 = np.transpose(uniform_direction(I).rvs(1))
    x=np.vstack((l0,v0)).reshape(-1)
    # optimizition
    es = minimize(sumofsquare, x, method='Powell')
    # recover last column of A
    v=es.x[J:]/np.linalg.norm(es.x[J:])
    #recover A up to permutation and sign
    estimateA=np.hstack((cols,v.reshape(-1,1)))
    return estimateA

In [105]:
# function testing how well is the approximation
def similarity_measures(B,A):
    J=A.shape[1]
    columnlist=[]
    for i in range(J):
        v=A[:,i]
        dislist=[]
        signlist=[]
        for j in range(B.shape[1]):
            u=abs(v-B[:,j])
            uprime=abs(v+B[:,j])
            sign=1
            if np.sum(u)<np.sum(uprime):
                dislist.append(np.sum(u))
                signlist.append(sign)
            else:
                sign=-1
                dislist.append(np.sum(uprime))
                signlist.append(sign)
        a=min(dislist)
        index=dislist.index(a)
        sign=signlist[index]
        columnlist.append(sign*B[:,index])
        B=np.delete(B,index,1)
    newAes=np.transpose(np.vstack(tuple(columnlist)))
    C= newAes-A
    relfroberror=(np.sum(C*C)/J)**0.5
    froberror=(np.sum(C*C))**0.5
    cosine_similarity=np.mean(np.sum(newAes*A,axis=0))
    last_column_cosine_similarity=np.sum(newAes[:,-1]*A[:,-1])
    # for now the bound is 0.95
    if last_column_cosine_similarity>=0.95:
        succesfulflag_last_column=1
    else:
        succesfulflag_last_column=0
    return last_column_cosine_similarity,cosine_similarity,relfroberror,froberror,succesfulflag_last_column,newAes,A


#apply algorithm
def algorithm(I,J,samplesize):
    (A,data)=observeddata(samplesize,I,J)
    estimateA=recovermixingmatrix(I,J,data)
    return similarity_measures(estimateA,A)

def algorithm_return_relfroberror(I,J,samplesize):
    (A,data)=observeddata(samplesize,I,J)
    estimateA=recovermixingmatrix(I,J,data)
    last_column_cosine_similarity,cosine_similarity,relfroberror,froberror,succesfulflag_last_column,newAes,A=similarity_measures(estimateA,A)
    return relfroberror

def algorithm_allJ(I,Jlist,samplesize):
    last_row_cosine_similaritylist=[]
    relative_frobneuserror_list=[]
    frobneuserror_list=[]
    successful_list=[]
    for J in Jlist:
        (last_column_cosine_similarity,cosine_similarity,relfroberror,froberror,succesfulflag_last_column,newAes,A)=algorithm(I,J,samplesize)
        last_row_cosine_similaritylist.append(last_column_cosine_similarity)
        relative_frobneuserror_list.append(relfroberror)
        frobneuserror_list.append(froberror)
        successful_list.append(succesfulflag_last_column)
    print(0)
    return last_row_cosine_similaritylist,relative_frobneuserror_list,frobneuserror_list,successful_list


In [107]:
Iequals4Jlist=list(range(2,13))
Iequals5Jlist=list(range(2,18))
Iequals6Jlist=list(range(2,24))
Iequals7list=list(range(2,31))
Iequals8list=list(range(2,39))

def plot_graphs(I,Jlist,n_iter,samplesize,n_jobs):
    output=Parallel(n_jobs=n_jobs)(delayed(algorithm_allJ)(I,J,samplesize) for I,J,samplesize in [(I,Jlist,samplesize)]*n_iter)
    last_column_cosine_similaritylistlist,relfroberrorlistlist,froberrorlistlist,succesfulflag_last_columnlistlistlist=zip(*output)
    last_row_cosine_similarityarry=np.array(last_column_cosine_similaritylistlist)
    relative_frobneuserror_listarray=np.array(relfroberrorlistlist)
    frobneuserror_listarray=np.array(froberrorlistlist)
    successful_listarray=np.array(succesfulflag_last_columnlistlistlist)
    rel_frob_mean=np.mean(relative_frobneuserror_listarray,axis=0)
    fig,((ax1,ax2),(ax3,ax4),(ax5,ax6))=plt.subplots(3,2,figsize=(36,36))
    ax1.boxplot(relative_frobneuserror_listarray)
    ax2.boxplot(last_row_cosine_similarityarry)
    ax3.boxplot(frobneuserror_listarray)
    ax4.bar(Jlist,np.sum(successful_listarray/n_iter,axis=0))
    ax1.set_xticklabels(Jlist)
    ax2.set_xticklabels(Jlist)
    ax3.set_xticklabels(Jlist)
    ax4.set_xticks(Jlist)
    ax1.set_title(f'relative_frobneuserror')
    ax2.set_title(f'last_column_cosine_similarity')
    ax3.set_title(f'frobeuserror')
    ax4.set_title(f'successful_rate')
    ax1.axvline(x=I*(I-1)/2-0.5,linestyle='--')
    ax2.axvline(x=I*(I-1)/2-0.5,linestyle='--')
    ax3.axvline(x=I*(I-1)/2-0.5,linestyle='--')
    ax4.axvline(x=I*(I-1)/2+0.5,linestyle='--')
    ax5.axvline(x=I*(I-1)/2,linestyle='--')
    ax5.plot(Jlist,rel_frob_mean)
    ax5.set_title('rel_frob_mean')
    ax5.set_xticks(Jlist)
    plt.savefig(f'I={I},n_iter={n_iter},samplesize={samplesize}')



In [ ]:
I=4
Jlist=Iequals4Jlist
n_iter=1000
samplesize=100000
plot_graphs(I,Jlist,n_iter,samplesize,10)

In [ ]:
I=5
Jlist=Iequals5Jlist
n_iter=1000
samplesize=100000
plot_graphs(I,Jlist,n_iter,samplesize,20)

In [ ]:
I=6
Jlist=Iequals6Jlist
n_iter=1000
samplesize=100000
plot_graphs(I,Jlist,n_iter,samplesize,20)

In [ ]:
I=7
Jlist=Iequals7list
n_iter=1000
samplesize=100000
plot_graphs(I,Jlist,n_iter,samplesize,20)

In [ ]:
I=8
Jlist=Iequals8list
n_iter=1000
samplesize=100000
plot_graphs(I,Jlist,n_iter,samplesize,20)